In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "source": [
    "# Notebook clínico maestro\n",
    "Comparación de EEGNet y ShallowConvNet en dataset clínico (CN/MCI/AD + tau).\n",
    "\n",
    "Incluye:\n",
    "- Carga de datos clínicos\n",
    "- Entrenamiento paralelo de EEGNet y ShallowConvNet\n",
    "- Evaluación y métricas\n",
    "- Visualización comparativa"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 1: Librerías\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.optim as optim\n",
    "from torch.utils.data import TensorDataset, DataLoader\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import yaml\n",
    "from src.utils import load_clinico_full\n",
    "from src.models import EEGNet, ShallowConvNet\n",
    "from src.metrics import log_metrics"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 2: Configuración\n",
    "with open(\"configs/clinico.yaml\", \"r\") as f:\n",
    "    config = yaml.safe_load(f)\n",
    "\n",
    "epochs = config[\"training\"][\"epochs\"]\n",
    "batch_size = config[\"training\"][\"batch_size\"]\n",
    "learning_rate = config[\"training\"][\"learning_rate\"]\n",
    "dropout_eegnet = config[\"eegnet\"][\"dropout\"]\n",
    "dropout_shallow = config[\"shallowconvnet\"][\"dropout\"]\n",
    "segment = config[\"general\"][\"segment\"]\n",
    "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 3: Cargar datos clínicos\n",
    "X_tr, y_tr, X_val, y_val, chans, samples = load_clinico_full(\n",
    "    base_path=\"data/raw/CLINICO\",\n",
    "    apply_filter=config[\"general\"][\"apply_filter\"],\n",
    "    segment=segment\n",
    ")\n",
    "X_tr, y_tr = X_tr.to(device), y_tr.to(device)\n",
    "X_val, y_val = X_val.to(device), y_val.to(device)\n",
    "\n",
    "train_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True)\n",
    "val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 4: Definir modelos\n",
    "model_eegnet = EEGNet(n_channels=chans, n_times=samples, n_classes=3, dropout=dropout_eegnet).to(device)\n",
    "model_shallow = ShallowConvNet(n_channels=chans, n_times=samples, n_classes=3, dropout=dropout_shallow).to(device)\n",
    "\n",
    "criterion = nn.CrossEntropyLoss()\n",
    "opt_eegnet = optim.Adam(model_eegnet.parameters(), lr=learning_rate)\n",
    "opt_shallow = optim.Adam(model_shallow.parameters(), lr=learning_rate)"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 5: Entrenamiento paralelo\n",
    "losses_eegnet, losses_shallow = [], []\n",
    "\n",
    "for epoch in range(epochs):\n",
    "    model_eegnet.train()\n",
    "    model_shallow.train()\n",
    "    train_losses_eegnet, train_losses_shallow = [], []\n",
    "\n",
    "    for xb, yb in train_loader:\n",
    "        # EEGNet\n",
    "        opt_eegnet.zero_grad()\n",
    "        out_eegnet = model_eegnet(xb)\n",
    "        loss_eegnet = criterion(out_eegnet, yb)\n",
    "        loss_eegnet.backward()\n",
    "        opt_eegnet.step()\n",
    "        train_losses_eegnet.append(loss_eegnet.item())\n",
    "\n",
    "        # ShallowConvNet\n",
    "        opt_shallow.zero_grad()\n",
    "        out_shallow = model_shallow(xb)\n",
    "        loss_shallow = criterion(out_shallow, yb)\n",
    "        loss_shallow.backward()\n",
    "        opt_shallow.step()\n",
    "        train_losses_shallow.append(loss_shallow.item())\n",
    "\n",
    "    losses_eegnet.append(np.mean(train_losses_eegnet))\n",
    "    losses_shallow.append(np.mean(train_losses_shallow))\n",
    "    print(f\"Epoch {epoch+1}, EEGNet Loss: {losses_eegnet[-1]:.4f}, ShallowConvNet Loss: {losses_shallow[-1]:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 6: Validación comparativa\n",
    "def evaluate(model, loader):\n",
    "    model.eval()\n",
    "    val_losses, val_preds, val_targets = [], [], []\n",
    "    with torch.no_grad():\n",
    "        for xb, yb in loader:\n",
    "            out = model(xb)\n",
    "            val_loss = criterion(out, yb)\n",
    "            val_losses.append(val_loss.item())\n",
    "            _, preds = torch.max(out, 1)\n",
    "            val_preds.extend(preds.cpu().numpy())\n",
    "            val_targets.extend(yb.cpu().numpy())\n",
    "    acc = (np.array(val_preds) == np.array(val_targets)).mean()\n",
    "    return np.mean(val_losses), acc\n",
    "\n",
    "val_loss_eegnet, val_acc_eegnet = evaluate(model_eegnet, val_loader)\n",
    "val_loss_shallow, val_acc_shallow = evaluate(model_shallow, val_loader)\n",
    "\n",
    "print(f\"EEGNet -> Val Loss: {val_loss_eegnet:.4f}, Val Acc: {val_acc_eegnet*100:.2f}%\")\n",
    "print(f\"ShallowConvNet -> Val Loss: {val_loss_shallow:.4f}, Val Acc: {val_acc_shallow*100:.2f}%\")"
   ]
  },
  {
   "cell_type": "code",
   "source": [
    "# Sección 7: Visualización comparativa\n",
    "plt.figure(figsize=(10,5))\n",
    "plt.plot(range(1, epochs+1), losses_eegnet, label=\"EEGNet Loss\")\n",
    "plt.plot(range(1, epochs+1), losses_shallow, label=\"ShallowConvNet Loss\")\n",
    "plt.xlabel(\"Epochs\")\n",
    "plt.ylabel(\"Training Loss\")\n",
    "plt.title(\"Evolución de pérdidas en dataset clínico\")\n",
    "plt.legend()\n",
    "plt.show()\n",
    "\n",
    "plt.bar([\"EEGNet\", \"ShallowConvNet\"], [val_acc_eegnet*100, val_acc_shallow*100])\n",
    "plt.ylabel(\"Validation Accuracy (%)\")\n",
    "plt.title(\"Comparación de accuracy en dataset clínico\")\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


{'cells': [{'cell_type': 'markdown',
   'source': ['# Notebook clínico maestro\n',
    'Comparación de EEGNet y ShallowConvNet en dataset clínico (CN/MCI/AD + tau).\n',
    '\n',
    'Incluye:\n',
    '- Carga de datos clínicos\n',
    '- Entrenamiento paralelo de EEGNet y ShallowConvNet\n',
    '- Evaluación y métricas\n',
    '- Visualización comparativa']},
  {'cell_type': 'code',
   'source': ['# Sección 1: Librerías\n',
    'import torch\n',
    'import torch.nn as nn\n',
    'import torch.optim as optim\n',
    'from torch.utils.data import TensorDataset, DataLoader\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import yaml\n',
    'from src.utils import load_clinico_full\n',
    'from src.models import EEGNet, ShallowConvNet\n',
    'from src.metrics import log_metrics']},
  {'cell_type': 'code',
   'source': ['# Sección 2: Configuración\n',
    'with open("configs/clinico.yaml", "r") as f:\n',
    '    config = yaml.safe_load(f)\n',
    '\n',
    